# An Analysis of State Relevance on International Affairs through their Relevance in Armed Conflict

## A.

Analyses of states in terms of their international relevance usually revolve traditional analyses of measures such as their economy, military power, and diplomatic endeavors. This however does not take into account their relation in armed conflict around the world. Participation in conflict is arguably an important measure to take into account. Direct analysis of individual conflicts however is not enough and will not reflect that conflict and the states' involved impact on the international stage.

## B.
Using the PageRank algorithm we can create a network of states connected via their history of conflict between other states. Through this we can use the PageRank algorithm to determine a state's importance overall. What is solved here is that we do not only analyze local relevance, but rather the importance of state globally.

## C.

As mentioned, this will help in providing quantifiable data on a state's global relevance. Through this project we can find states that have the most relevance in terms of armed conflict. Hopefully the data found in this project will coincide with other metrics, as well as provide new insight.

In [ ]:
import os
import pandas as pd
import numpy as np
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

zip_url = 'https://ucdp.uu.se/downloads/ucdpprio/ucdp-prio-acd-251-csv.zip'
zip_filename = 'ucdp-prio-acd-251-csv.zip'
csv_filename = 'UcdpPrioConflict_v25_1.csv'
useful_columns = ['side_a', 'side_a_id', 'side_b', 'side_b_id']

df = None
try:
    # Download zip, store as bytes object
    zip_data = None
    with urlopen(zip_url) as zip_response:
        zip_data = BytesIO(zip_response.read())

    # Extract csv from zip, store as dataframe
    with ZipFile(zip_data, 'r') as zip_file:
        with zip_file.open(csv_filename) as csv_file:
            df = pd.read_csv(csv_file, usecols=useful_columns)

    print(df.head())

except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Get ids and assemble a list of all links
ids_to_indices_dict = {}
name_list = []
linked_pairs = []
for row in df.itertuples():
  # Parse str data from csv
  side_a_id_list = [int(id_str.strip()) for id_str in row.side_a_id.split(',')]
  side_b_id_list = [int(id_str.strip()) for id_str in row.side_b_id.split(',')]

  side_a_name_list = [name_str.strip() for name_str in row.side_a.split(',')]
  side_b_name_list = [name_str.strip() for name_str in row.side_b.split(',')]

  for i in range(len(side_a_id_list)):
      id_a = side_a_id_list[i]
      if id_a not in ids_to_indices_dict:
          ids_to_indices_dict[id_a] = len(ids_to_indices_dict)
          name_a = side_a_name_list[i]
          name_list.append(name_a)

      for j in range(len(side_b_id_list)):
          id_b = side_b_id_list[j]
          if id_b not in ids_to_indices_dict:
              ids_to_indices_dict[id_b] = len(ids_to_indices_dict)
              name_b = side_b_name_list[j]
              name_list.append(name_b)

          index_a = ids_to_indices_dict[id_a]
          index_b = ids_to_indices_dict[id_b]

          linked_pairs.append((index_a, index_b))
# print(linked_pairs)
# print(ids_to_indices_dict)
# print(len(linked_pairs))
# print(len(ids_to_indices_dict))

# Assemble adjacency matrix using list of links.
n = len(ids_to_indices_dict)
adj_matrix = np.zeros((n, n), dtype=int)
for pair in linked_pairs:
  # Boolean OR both sides of the link to 1.
  adj_matrix[pair[0], pair[1]] = adj_matrix[pair[0], pair[1]] | 1
  adj_matrix[pair[1], pair[0]] = adj_matrix[pair[1], pair[0]] | 1
print(adj_matrix)

In [ ]:
def compute_pagerank(adj_matrix, damping=0.85, max_iters=10000, tolerance=1e-6):
    # Normalize the adjacency matrix to ensure that each column sums to 1
    adj_matrix = adj_matrix / adj_matrix.sum(axis=0, keepdims=True)

    num_pages = adj_matrix.shape[0]

    # Initialize the PageRank vector
    pagerank = np.ones(num_pages) / num_pages

    # Calculate pagerank score iteratively until it converges within tolerance
    for iter in range(max_iters):
        numerator = 1.0 - damping
        denominator = num_pages + damping * adj_matrix @ pagerank
        new_pagerank = numerator / denominator

        # Check for convergence
        if np.linalg.norm(new_pagerank - pagerank, 1) < tolerance:
            break

        pagerank = new_pagerank

    return pagerank

result = compute_pagerank(adj_matrix)

In [ ]:
# Display results
sum = 0
for i in range(n):
  print(f'{name_list[i]}: {result[i]}')
  sum += result[i]
print(f'\nOverall sum: {sum}')